# Data handling with python



In this session, we will learn how to use Python to explore and analyse data. We will start by first using basic Python to read and write data from a file, followed by using built-in Python libraries for simple data analysis. We will then use the `pandas` Python library to perform advanced data exploration and manipulation.

## Reading files

The main dataset we will use in today's session is the United States Department of Agriculture’s Food Composition Database, which contains data on various type of food along with their nutrient information.

We will first load a small portion of data stored in `data/food_tiny.csv` file. To do this in python, we need to open the data and read in the content of the data, either all at once or line by line. The `with` statement is used to make sure that the file is properly closed once we have finished reading the file.

In [ ]:
# Reading from file
with open("data/food_tiny.csv") as f:
    for line in f:
        print(line.strip())

Let's extract a couple of columns from the data. Notice that the data is formatted as a CSV (comma-separated value) file, where each column entries are separated by comma.

In [ ]:
# Printing only the food description and lipid content
with open("data/food_tiny.csv") as f:
    for line in f:
        data = line.strip().split(",")
        print(data[1], data[23])

Our output seems incorrect. We are only getting a partial content for the description column and the column for total lipid content is showing data for saturated fat content instead!

This is because we have comma in the description columns (e.g. `Milk, whole`), which means that splitting by comma will result in incomplete column separation. If you look at the unprocessed file content above, notice that the value for each column is actually enclosed within double-quotes (`"`) - this is part of the CSV format specification to handle values which contains comma.

In order to extract each column properly, we will need to split by comma surrounded by double quotes.

In [ ]:
# Correctly printing only the food description and lipid content
with open("data/food_tiny.csv") as f:
    for line in f:
        data = line.strip().split('","') # Split the column by comma surrounded by double quote
        print(data[1], ":", data[23])

We can write to a file by using the same `open()` function and passing the `w` argument to indicate that you want to open the file in write mode. However, this will erase the content of any existing file, so if you want to add to a file, you will want to pass the `a` argument to indicate you want to open the file in append mode instead.

In [ ]:
# Writing the food description and lipid content to a file
food_lipid_content_info = []
with open("data/food_tiny.csv") as f:
    for line in f:
        data = line.strip().strip('"')
        data = data.split('","')
        food_lipid_content_info.append([data[1], data[23]])

with open("data/food_lipid_content_tiny.tsv", "w") as o:
    for record in food_lipid_content_info:
        o.write('\t'.join(record) + "\n")

**Practice**: Use the `food_tiny.csv` file to create a new file containing the food description and the vitamin contents of the food, together with a new column containing the total vitamin contents of the food.

In [ ]:
# Solution 1
food_vitamin_content_info = []
with open("data/food_tiny.csv") as f:
    line_count = 0
    for line in f:
        data = line.strip().strip('"')
        data = data.split('","')

        if line_count == 0:
             new_column = "Data.Vitamins.Total Content"
        else:
            vitamin_content = 0.0
            for i in range(32, 38):
                vitamin_content += float(data[i])
            new_column = str(vitamin_content)

        data.append(new_column)
        food_vitamin_content_info.append(data)
        line_count += 1

with open("data/food_extended_tiny.tsv", "w") as o:
    for record in food_vitamin_content_info:
        o.write('\t'.join(record) + "\n")

In [ ]:
# Solution 2 - using Python list comprehension
food_vitamin_content_info = []
with open("data/food_tiny.csv") as f:
    for line_count, line in enumerate(f):
        data = line.strip().strip('"').split('","')

        if line_count == 0:
            food_vitamin_content_info.append(data + ["Data.Vitamins.Total Content"])
        else:
            vitamin_content = sum([float(data[i]) for i in range(32, 38)])
            food_vitamin_content_info.append(data + [str(vitamin_content)])

with open("data/food_extended_tiny.tsv", "w") as o:
    for record in food_vitamin_content_info:
        o.write('\t'.join(record) + "\n")

## Modules

The base Python program only contains generic functionality, which you can extend by importing modules (or libraries) which implement specific functionality. The default Python installation contains a number of standard library designed to cover a broad range of use case, but you can also install external modules to further extend the functionality of Python.

One of the standard library included in Python is the `csv` module designed to handle reading and writing of CSV (and CSV-like tabular) files. As you saw above, dealing with CSV files can be tricky and writing your own parser for CSV files may result in incorrectly parsed file due to the different variation in CSV file content. In order to make sure we can correctly parse any CSV file properly, we will use reading and writing function implemented in the `csv` module by `import`-ing the module.

In [ ]:
import csv

The `csv` module provide a few different method to read and write CSV files, as detailed in the [module documentation](https://docs.python.org/3/library/csv.html). We will be using the `DictReader()` and `DictWriter()` functions, which will allow us to work with the data in dictionary form. In order to use the function from the module, we will need to use `<module>.<function>()` syntax.

In [ ]:
# Reading file using csv module
data = []
with open("data/food_tiny.csv") as f:
    reader = csv.DictReader(f, delimiter = ",")
    for row in reader:  
        data.append(row)

data[0:2]

We store each row of data (in the form of a dictionary object) into a list. Note that the dictionary object returned by `DictReader()` is an [ordered dictionary](https://docs.python.org/3/library/collections.html#ordereddict-objects), which remember the order that the items are inserted.

In [ ]:
for d in data:
    print(d['Description'], ":", d['Data.Fat.Total Lipid'])

In [ ]:
# Calculate total Vitamin content
for d in data:
    d['Data.Vitamins.Total Vitamin'] = float(d['Data.Vitamins.Vitamin A - RAE']) + float(d['Data.Vitamins.Vitamin B12']) + float(d['Data.Vitamins.Vitamin B6']) + float(d['Data.Vitamins.Vitamin C']) + float(d['Data.Vitamins.Vitamin E']) + float(d['Data.Vitamins.Vitamin K'])

data[0:2]

In [ ]:
# Writing the updated data to a new file
with open("data/food_extended_tiny.csv", "w") as f:
    writer = csv.DictWriter(f, data[0].keys(), delimiter=',')
    writer.writeheader() # write header

    for d in data:
        writer.writerow(d) # write row

Python standard library also contains modules such as the `statistics` module which are useful for data analysis. Let's use the `mean()` function from the `statistics` module to calculate the mean calcium content of milk.

In [ ]:
import statistics

We can list the functions available in the statistics module by using the `dir()` function.

In [ ]:
dir(statistics)

We can also directly import specific function from a module using the `from <module> import <functions>` syntax. We can then use this function directly without needing to use the `<module>.<function>` syntax as before.

In [ ]:
# Calculate average calcium content
from statistics import mean, median

calcium_content = []

for d in data:
    calcium_content.append(float(d['Data.Major Minerals.Calcium']))

mean(calcium_content)

## Pandas

The `pandas` module is an external Python library for data analysis and manipulation of data. It is primarily designed to handle tabular data using the `DataFrame` object, which is itself composed of `Series` 'column' objects to represent one dimensional data. The module provides tools for reading and writing data from various format, clean and preprocess data, handle missing values, aggregate and transform data, as well as merging and joining datasets.

In order to be able to use `pandas` in our Python program, we will need to install it first using `pip install pandas`. We can then import it as before using the `import` command, though this time we will use the `import <module> as <alias>` syntax to allow us to use the alias `pd` to refer to the `pandas` module.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 15)

The `pandas` module contains functions for reading and writing file from various formats, such as CSV, xls, json, and SQL. We will use the `read_csv()` function from `pandas` to read the full Food Composition Database file stored in `data/food.csv`, which will return a `DataFrame` object.

In [ ]:
# Reading file using pandas
nutrition = pd.read_csv("data/food.csv")
nutrition

Looking at the `DataFrame` above, we can see that the rows represent the **observations** (entries for food item), while the columns represent the observed **variables** (Category, Description, Nutrient Data Bank Number and so on).

By default, pandas will assign a positional-based numeric index for each rows, starting from 0, as shown in the leftmost column. We can specify a custom index using one of the column from the data by passing the name of the column to the `index_col` argument in the `read_csv()` function.

In [ ]:
# Specifying index column
nutrition = pd.read_csv("data/food.csv", index_col="Nutrient Data Bank Number")
nutrition

### Exploring data

A `pandas` `DataFrame` is a 2-dimensional object which is composed of columns with different data types, such as string, boolean, integer, float, categorical/factor and more. This is similar to a spreadsheet, an SQL table or `data.frame` object in R. Both the rows and columns of `DataFrame` are indexed and can be accessed using either number or name.

Let's start with looking at the basic information of our Food Composition Database. We do this by using the `DataFrame` object **methods** `.info()`. A **method** is essentially a function which are specific to an object and can only be used on an instance of the object.

In [ ]:
nutrition.info()

The output from `.info()` method shows the object type `pandas.core.frame.DataFrame`, the index of the `DataFrame` object, the information on columns within the `DataFrame` object as well as the memory usage of the `DataFrame` object.

We can also query some information about our `DataFrame` object by looking at the object **attributes**. An object **attributes** contains information of about an object and can be accessed using `<object>.<attribute>` syntax. Note that unlike a method, we do not use `()` to access the object's attribute.

Some attributes provided by the `DataFrame` object includes the types of data contained in the object, accessible using `.dtypes` attribute, and the dimension of the `DataFrame` object in (row, column) format, using the `.shape` attribute.

In [ ]:
nutrition.dtypes

In [ ]:
nutrition.shape # returns (#row, #column)

Another way to  check the number of rows is using the `len()` function.

In [ ]:
len(nutrition)

The names of the rows and columns of the `DataFrame` object can be accessed using the attributes `.index.values` and `.columns.values`.

In [ ]:
nutrition.index.values

In [ ]:
nutrition.columns.values

 We can have a peek at the top and bottom rows of the `DataFrame` object using the `.head()` and `.tail()` methods. By default, this will return the first or last 5 rows only, though you can specify the number of rows returned by passing the number of rows as an argument.

In [ ]:
# Look at the top rows
nutrition.head()

In [ ]:
# Look at the bottom rows
nutrition.tail()

In [ ]:
# Look at the bottom 10 rows
nutrition.tail(10)

We can also use the `.describe()` method to compute some summary statistics for columns. Since our `DataFrame` object contains a mix of categorical and numeric columns, the `.describe()` will return summary statistics on the numeric columns only.

In [ ]:
nutrition.describe()

We can do this on each column individually by using methods such as `.mean()` and `.std()` on each column.

In [ ]:
# Finding mean α-Carotene content
nutrition["Data.Alpha Carotene"].mean()

In [ ]:
# Finding standard deviation of α-Carotene content
nutrition["Data.Alpha Carotene"].std()

For columns containing categorical variables, such as Category and Description, we can use methods such as `.unique()` to show the unique values for the columns and `.value_counts()` to calculate the frequency of each unique values.

In [ ]:
# Show all unique values in Category column
nutrition["Category"].unique()

In [ ]:
# Show the frequency of each unique values in Category column
nutrition["Category"].value_counts()

### Selecting columns and rows

Selecting columns and rows can be done using either **positional** indexing, which uses the numerical position of the rows, or **label-based** indexing, which uses the column or row index names. Similar to selecting items in list, we use the square brackets `[]` to select columns and/or rows.

In [ ]:
# Selecting the Description column
nutrition['Description']

In [ ]:
# Selecting the top 5 rows
nutrition[:5]

We can select both rows and columns together by selecting the row, followed by selecting the column, using the `[<row>][<column>]` syntax.

In [ ]:
nutrition[:5]['Description']

We can also select both rows and columns together with **positional** indexing using the `.iloc[<row>, <column>]` syntax.

In [ ]:
nutrition.iloc[:5,1]

To do the row and column selection using **label-based** indexing, we will need to use the `.loc[<row names>, <column names>]` syntax.

In [ ]:
nutrition.loc[11000000, "Description"]

We can select multiple rows and/or column by passing a list in both `.iloc[]` and `.loc[]` syntax.

In [ ]:
nutrition.iloc[[1,2], [0,1]]

In [ ]:
nutrition.loc[[11000000, 11100000], ["Category", "Description"]]

We can remove an index from the `DataFrame` object by using the `.reset_index()` method to revert to the default positional numeric index. We can also assign an index manually using the `.set_index()` method, with the column name to be used as the index as the argument.

In [ ]:
# Removing the existing index based on the Nutrient Data Bank Number
nutrition = nutrition.reset_index()
nutrition

In [ ]:
# Re-assigning the index back to the Nutrient Data Bank Number
nutrition = nutrition.set_index("Nutrient Data Bank Number")
nutrition

### Filtering rows



We can filter for rows in the `DataFrame` object which matches some specific criteria based on the value of the columns. This is done by specifying a condition within the square brackets `[]`, which will effectively produce a boolean (True/False) index mask to select rows.

In [ ]:
# Selecting rows with category Rice only
nutrition[nutrition["Category"] == "Rice"]

In [ ]:
# Selecting rows where alpha carotene values is greater than 5
nutrition[nutrition["Data.Alpha Carotene"] > 5]

We can combine multiple condition using boolean operators. Unlike in base Python, we will need to enclose each condition in brackets `()` and use the following symbols for the boolean operators: `&` for `and`, `|` for `or` and `~` for `not`.

In [ ]:
nutrition[(nutrition["Category"] == "Rice") & (nutrition["Data.Alpha Carotene"] > 5)]

We can filter for multiple values for a given column using the `.isin()` method.

In [ ]:
nutrition[nutrition["Category"].isin(["Bread", "Rice"])]

For column containing strings, we can use string operations for filtering as well. For example, if we want to select for all category which contains the string 'milk', we can use the `.str.contains()` method.

In [ ]:
nutrition[nutrition["Category"].str.contains('milk', case=False)] # We set case argument to False to do case insensitive search

### Manipulating data

We can add a new column to the existing `DataFrame` object by using the `<DataFrame_object>[<new_column>] = <column_value>` syntax. In this example, we will create a new column to store the total vitamin contents (in milligrams) based on the value of the vitamin-related columns. Some of the vitamins (Vitamin A, B12 and K) are measured in micrograms, which meant that we will need to convert them to milligrams by dividing the value of the columns by 1000.

In [ ]:
# Adding a new column to store total vitamin contents
nutrition['Data.Vitamins.Total Vitamin'] = nutrition['Data.Vitamins.Vitamin A - RAE']/1000 + nutrition['Data.Vitamins.Vitamin B12']/1000 + nutrition['Data.Vitamins.Vitamin B6'] + nutrition['Data.Vitamins.Vitamin C'] + nutrition['Data.Vitamins.Vitamin E'] + nutrition['Data.Vitamins.Vitamin K']/1000
nutrition

We can sort the `DataFrame` object based on the value of the column using `.sort_values()` method. Note that unlike sorting a list, this will does not change the existing `DataFrame` object by default and it will return a new sorted `DataFrame` object instead. We can change this behaviour by setting the `inplace` argument to `True`.

In [ ]:
# Sorting data by Category
nutrition_sorted = nutrition.sort_values('Category')
nutrition_sorted

In [ ]:
# Sorting data by Category in reverse on the existing object
nutrition_sorted.sort_values('Data.Fiber', ascending=False, inplace=True)
nutrition_sorted

### Grouping data

We can perform grouping of data in order to calculate some aggregated values for each group. Let's say that we want to calculate the mean nutrient content for each food category type, we can do this by first using the `.groupby()` method and specifying the columns we want to group the data as the arguments for the method.

In [ ]:
# Grouping by category
nutrition.groupby('Category')

The `.groupby()` method returns a `DataFrameGroupBy` object that we can apply aggregation operations on. Let's calculate the mean for each column within each group using the `.mean()` method.

In [ ]:
# Calculate mean of each column per food category
nutrition_averaged = nutrition.groupby('Category').mean(numeric_only=True) # Need to include numeric_only otherwise it will throw error when handling columns with string
nutrition_averaged

You can see the list of aggregation methods provided by pandas in the [user guide](https://pandas.pydata.org/docs/user_guide/groupby.html#built-in-aggregation-methods).

### Merge and join

We can combine multiple `DataFrame` object using the `concat()` function from `pandas`.

In [ ]:
nutrition_rice = nutrition[nutrition["Category"]=="Rice"]
nutrition_bread = nutrition[nutrition["Category"]=="Bread"]

pd.concat([nutrition_rice,nutrition_bread])

We can also join datasets using the `merge()` function.

To illustrate the join functionality, let's first load a dataset of weekly fruit consumption stored in `data/weekly_fruit_consumption.csv`. We want to join this fruit consumption data with the averaged nutrition data in order to calculate the nutrition of fruits consumed by each person.

In [ ]:
fruit_consumption = pd.read_csv("data/weekly_fruit_consumption.csv")
fruit_consumption

In the fruit consumption data, we have each column representing the day of the week and each row representing the fruits consumed by each person over the week.

In order to join this data with the nutrition data to get the nutrition information for each fruit, we need to change the 'shape' of the data so that each row represent the fruit consumed by each person on a single day of the week. To do this, we will use the `.melt()` method.

In [ ]:
fruit_consumption_melted = fruit_consumption.melt(id_vars="Name", var_name="Day", value_name="Fruit")
fruit_consumption_melted

If we want to revert this back to the previous data 'shape', we can use the `.pivot()` method.

In [ ]:
fruit_consumption_melted.pivot(columns="Day", index="Name", values="Fruit")

Let's now perform join between the fruit consumption data with the nutrition data.

In [ ]:
pd.merge(fruit_consumption_melted, nutrition, left_on="Fruit", right_on="Category")

### Missing data

Pandas provide functionality to handle missing data, which is primarily represented as `NaN` and are by default excluded from analysis. We can use the `.info()` method to find columns containing missing data.

In [ ]:
fruit_consumption.info()

To find the location of the missing data, we can use either `pandas.isna()` function or `.isnull()` method.

In [ ]:
pd.isna(fruit_consumption)

In [ ]:
fruit_consumption.isnull()

We can also remove any rows containing missing data using the `.dropna()` method. We can further specify if you want to drop rows containing missing data in certain columns only by passing the name of those columns in the `subset` argument.

In [ ]:
# Drop any rows which contain NA in any of the column
fruit_consumption.dropna()

In [ ]:
# Drop any rows which contain NA in the Saturday column only
fruit_consumption.dropna(subset = ["Saturday"])

If we want to instead fill in the missing data, we can use the `.fillna()` method.

In [ ]:
fruit_consumption.fillna(value={"Wednesday":"Apple", "Saturday":"Guava", "Sunday":"Peach"})

## Exercise

1. Using the weekly fruit consumption data, find the most popular fruit.
2. Using the weekly fruit consumption data, calculate the amount and variety of fruit eaten by each person. Who ate the least amount of fruit and who ate the most variety of fruit?
3. Using the weekly fruit consumption data with the USDA Food Composition Database, find the total amount of fiber consumed by each person. Note that the fruit consumed is fresh fruit.
4. Using the weekly fruit consumption data with the USDA Food Composition Database, find the average daily nutrients consumed by each person. Write this information into a file called "daily_nutrition_consumption.csv". You will need to consult `pandas` documentation for this.

In [ ]:
# Find the most popular fruit
fruit_consumption_melted["Fruit"].value_counts().sort_values(ascending=False)

In [ ]:
# Calculate the amount and variety of fruit eaten by each person
fruit_consumption_melted.groupby("Name").count()["Fruit"]
fruit_consumption_melted.groupby("Name").nunique()["Fruit"]
fruit_consumption_melted.groupby("Name").agg(["count", "nunique"])["Fruit"]
fruit_consumption_melted.groupby("Name").agg(["count", "nunique"])["Fruit"]["count"].sort_values() # Least amount of fruit
fruit_consumption_melted.groupby("Name").agg(["count", "nunique"])["Fruit"]["nunique"].sort_values(ascending=False) # Most variety of fruit

In [ ]:
# Find the total amount of fiber consumed by each person per week
fruit_consumption_nutrition = pd.merge(fruit_consumption_melted, nutrition, left_on="Fruit", right_on="Category")
fruit_consumption_nutrition[fruit_consumption_nutrition["Name"]=="Alex"]

In [ ]:
# Find the average amount of fiber consumed by each person per day of the week
fruit_consumption_nutrition = pd.merge(fruit_consumption_melted, nutrition, how='left', left_on="Fruit", right_on="Category")
fruit_consumption_nutrition = fruit_consumption_nutrition[fruit_consumption_nutrition["Description"].str.contains("raw", na=True)]
fruit_consumption_nutrition = fruit_consumption_nutrition.fillna(0)
daily_fruit_consumption_nutrition = fruit_consumption_nutrition.groupby("Name").mean(numeric_only=True)
daily_fruit_consumption_nutrition.to_csv("data/daily_nutrition_consumption.csv")

## Acknowledgement

The session material in this notebook is adapted from:
- The University of Cambridge [Data Science in Python course](https://github.com/pycam/python-data-science)
- AztraZeneca's Data Science Academy [Data Science in Python course](https://github.com/semacu/data-science-python)
- Pandas [documentation]((http://pandas.pydata.org/pandas-docs/stable/tutorials.html)

The United States Department of Agriculture’s Food Composition Database used in this session notebook is comes from [Food Vitamins, Minelas, Macronutrient dataset](https://www.kaggle.com/datasets/mexwell/food-vitamins-minerals-macronutrient) under GPL2 license.